In [41]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [42]:
sc

<SparkContext master=yarn appName=lab02_RysinN>

In [43]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("lab02_RysinN").getOrCreate()

In [44]:
import re
import json
from functools import partial

import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql.window import Window
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StopWordsRemover
from pyspark.ml import Pipeline

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark.ml.param import Param, Params, TypeConverters

from pyspark import keyword_only

In [45]:
path_to_data = '/labs/slaba02/DO_record_per_line.json'

In [46]:
# Вариант 10
courses_to_make_recommendation = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], 
 [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], 
 [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], 
 [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], 
 [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], 
 [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [47]:
courses_to_make_recommendation

[[23126, 'en', 'Compass - powerful SASS library that makes your life easier'],
 [21617, 'en', 'Preparing for the AP* Computer Science A Exam — Part 2'],
 [16627, 'es', 'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'],
 [11556,
  'es',
  'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'],
 [16704, 'ru', 'Программирование на Lazarus'],
 [13702, 'ru', 'Математическая экономика']]

In [48]:
id_courses_for_recommendation = list(map(lambda x: x[0], courses_to_make_recommendation))

In [49]:
id_courses_for_recommendation

[23126, 21617, 16627, 11556, 16704, 13702]

In [50]:
data_df = spark.read.json(path_to_data)

In [51]:
data_df.show(5, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [52]:
#tokenizer = Tokenizer(inputCol="desc", outputCol="desc_words")

In [53]:
#@f.pandas_udf(returnType=t.ArrayType(t.StringType()))
#def clear_string(series):
#    regex = re.compile(u'[\w\d]{2,}', re.U)
#    words = series.str.findall(regex)
#    return words

# tokenized_data = data_df.withColumn("desc_words", clear_string("desc"))

In [54]:
def tokenize(series, regex):
        return series.str.lower().str.findall(regex)
    
class RegexTransformer(Transformer, HasInputCol, HasOutputCol):
    
    regex = Param(Params._dummy(), "regex",
                  "Python regular expression to match tokens",
                  typeConverter=TypeConverters.toString)
    
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, regex="\w+"):
        super(RegexTransformer, self).__init__()
        if inputCol is not None:
            self.setInputCol(inputCol)
        if outputCol is not None:
            self.setOutputCol(outputCol)
        self._set(regex=regex)
            
    def setRegex(self):
        self._set(regex=regex)
            
    def _transform(self, dataset):
        pattern = re.compile(self.getOrDefault("regex"), re.U)
        tokenize_udf = F.pandas_udf(partial(tokenize, regex=pattern), returnType=T.ArrayType(T.StringType()))
        return dataset.withColumn(self.getOutputCol(), tokenize_udf(self.getInputCol()))

In [55]:
tokenizer = RegexTransformer(inputCol="desc", outputCol="desc_words", regex=u'[\w\d]{2,}')

In [56]:
tokenizer.transform(data_df).show()

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|          desc_words|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[this, course, in...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[this, online, co...|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|[this, course, is...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|[we, live, in, di...|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|[this, self, pace...|
|9/humanities|15/m...|This game-based c...|  9|  en|College Foundatio...|Canvas Network|[this, game, base...|
|  14/soci

In [57]:
eng_stop_words = StopWordsRemover.loadDefaultStopWords("english")

In [58]:
remore_stopwords = StopWordsRemover(inputCol="desc_words", outputCol="words_filtered", stopWords=eng_stop_words)

In [59]:
hashingTF = HashingTF(inputCol="words_filtered", outputCol="rawFeatures", numFeatures=10000, binary=False)

In [60]:
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

In [61]:
pipeline = Pipeline(stages=[tokenizer, remore_stopwords, hashingTF, idf])

In [62]:
data_tr = pipeline.fit(data_df).transform(data_df)

In [63]:
data_tr.show(5, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [64]:
data_tr = data_tr.select("id", "lang", "name", "words_filtered", "features")

In [65]:
df_courses_for_recommendation = data_tr.filter(data_tr['id'].isin(id_courses_for_recommendation))

In [66]:
df_courses_for_recommendation.show()

+-----+----+--------------------+--------------------+--------------------+
|   id|lang|                name|      words_filtered|            features|
+-----+----+--------------------+--------------------+--------------------+
|11556|  es|Aprendizaje Colab...|[la, transformaci...|(10000,[249,522,5...|
|13702|  ru|Математическая эк...|[математическая, ...|(10000,[310,942,2...|
|16627|  es|Aprende Excel: Ni...|[hazte, más, empl...|(10000,[55,76,192...|
|16704|  ru|Программирование ...|[курсе, рассматри...|(10000,[381,1144,...|
|21617|  en|Preparing for the...|[introduction, co...|(10000,[17,161,36...|
|23126|  en|Compass - powerfu...|[improve, sass, s...|(10000,[87,246,25...|
+-----+----+--------------------+--------------------+--------------------+



In [67]:
df_courses_for_recommendation = df_courses_for_recommendation.select(F.col("id").alias("search_id"),
                                                                     F.col("lang").alias("search_lang"),
                                                                     F.col("name").alias("search_name"),
                                                                     F.col("features").alias("search_features")
                                                                    )

In [68]:
df_join = data_tr.join(F.broadcast(df_courses_for_recommendation),
                       (data_tr.lang == df_courses_for_recommendation.search_lang),
                      )\
                 .filter("id != search_id")

In [69]:
@F.udf(returnType=T.FloatType())
def cosine_similarity(x, y):
    return float(x.dot(y) / (x.norm(2) * y.norm(2)))

In [70]:
df_join = df_join.withColumn("cos_sim", 
                             cosine_similarity(F.col("features"), F.col("search_features"))
                            ).filter((~F.isnan("cos_sim")) & (~F.isnull("cos_sim"))
                                    )

In [71]:
df_join.show(5, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [72]:
window_search_id = Window.partitionBy("search_id").orderBy(F.col("cos_sim").desc())

In [73]:
sim_courses = df_join.withColumn("rank_similarity",
                                 F.row_number().over(window_search_id)
                                )\
                     .filter("rank_similarity < 10")\
                     .select('search_id', 'search_name', 'id', 'name', 'cos_sim', 'rank_similarity')

In [74]:
sim_courses_rdd = sim_courses.select(F.col('search_id').astype(F.StringType()), 'id').rdd

In [75]:
sim_courses_rdd.take(5)

[Row(search_id='23126', id=13665),
 Row(search_id='23126', id=14760),
 Row(search_id='23126', id=13782),
 Row(search_id='23126', id=20638),
 Row(search_id='23126', id=24419)]

In [76]:
answer = dict(sim_courses_rdd.groupByKey().mapValues(list).collect())

In [77]:
answer

{'16704': [1236, 1247, 1365, 1273, 20288, 1164, 8186, 1233, 8203],
 '11556': [16488, 468, 13461, 23357, 19330, 7833, 9289, 10447, 10384],
 '23126': [13665, 14760, 13782, 20638, 24419, 15909, 2724, 25782, 17499],
 '13702': [864, 21079, 8313, 1041, 28074, 8300, 13057, 1033, 21025],
 '16627': [11431, 12247, 17964, 5687, 11575, 17961, 12660, 25010, 5558],
 '21617': [21609, 21616, 22298, 21608, 21628, 21630, 21081, 21623, 19417]}

In [78]:
# Пишем в JSON
with open('lab02.json', 'w', encoding='utf-8') as f:
    json.dump(answer, f, indent=4)

In [79]:
!cat lab02.json

{
    "16704": [
        1236,
        1247,
        1365,
        1273,
        20288,
        1164,
        8186,
        1233,
        8203
    ],
    "11556": [
        16488,
        468,
        13461,
        23357,
        19330,
        7833,
        9289,
        10447,
        10384
    ],
    "23126": [
        13665,
        14760,
        13782,
        20638,
        24419,
        15909,
        2724,
        25782,
        17499
    ],
    "13702": [
        864,
        21079,
        8313,
        1041,
        28074,
        8300,
        13057,
        1033,
        21025
    ],
    "16627": [
        11431,
        12247,
        17964,
        5687,
        11575,
        17961,
        12660,
        25010,
        5558
    ],
    "21617": [
        21609,
        21616,
        22298,
        21608,
        21628,
        21630,
        21081,
        21623,
        19417
    ]
}

In [80]:
sc.stop()